In [7]:
# Install the required packages
%pip install pandas numpy

import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
df = pd.read_csv('data/PremierLeague.csv')
df = df.drop(labels=[
    'Time', 'HomeTeamCorners', 'AwayTeamCorners', 'HalfTimeHomeTeamGoals', 'HalfTimeAwayTeamGoals', 'HalfTimeResult', 'HomeTeamShots', 'AwayTeamShots', 'HomeTeamShotsOnTarget', 'AwayTeamShotsOnTarget', 'B365HomeTeam', 'B365AwayTeam', 'B365Draw', 'B365Over2.5Goals', 'B365Under2.5Goals', 'MarketMaxHomeTeam', 'MarketMaxDraw', 'MarketMaxAwayTeam', 'MarketAvgHomeTeam', 'MarketAvgDraw', 'MarketAvgAwayTeam', 'MarketMaxOver2.5Goals', 'MarketMaxUnder2.5Goals', 'MarketAvgOver2.5Goals', 'MarketAvgUnder2.5Goals', 'HomeTeamPoints', 'AwayTeamPoints'
    ],
    axis = 1,
    )
df = df.dropna()

In [ ]:
def get_referee_cards(referee_name, team_name):
    # Get matches where the referee officiated
    referee_matches = df[df['Referee'] == referee_name]
    
    # Get cards when team was home
    home_matches = referee_matches[referee_matches['HomeTeam'] == team_name]
    home_cards = home_matches['HomeTeamYellowCards'].sum() + home_matches['HomeTeamRedCards'].sum()
    
    # Get cards when team was away
    away_matches = referee_matches[referee_matches['AwayTeam'] == team_name]
    away_cards = away_matches['AwayTeamYellowCards'].sum() + away_matches['AwayTeamRedCards'].sum()
    
    total_matches = len(home_matches) + len(away_matches)
    total_cards = home_cards + away_cards
    
    return {
        'Total Cards': total_cards,
        'Matches Officiated': total_matches,
        'Cards per Match': total_cards/total_matches if total_matches > 0 else 0
    }

# Example usage - replace with your desired referee and team
referee = "Mike Dean"  # Example referee
team = "Arsenal"      # Example team
stats = get_referee_cards(referee, team)
print(f"\nCards given by {referee} to {team}:")
for key, value in stats.items():
    print(f"{key}: {value:.2f}")


Cards given by Mike Dean to Arsenal:
Total Cards: 5.00
Matches Officiated: 3.00
Cards per Match: 1.67


In [ ]:
def get_referee_cards(referee_name, team_name):
    referee_matches = df[df['Referee'] == referee_name]
    
    home_matches = referee_matches[referee_matches['HomeTeam'] == team_name]
    home_fouls = home_matches['HomeFouls'].sum()

    away_matches = referee_matches[referee_matches['AwayTeam'] == team_name]
    away_fouls = away_matches['AwayFouls'].sum()

    total_matches = len(home_matches) + len(away_matches)
    total_fouls = home_card + away_cards
    

In [15]:
referees = sorted(df['Referee'].unique())

# Print the list of referees
print("List of all referees:")
for referee in referees:
    print(referee)

# Optional: print total count
print(f"\nTotal number of referees: {len(referees)}")

List of all referees:
A D'Urso
A Madley
A Marriner
A Moss
A Taylor
A Wiley
A. G. Wiley
A. G. Wiley 
A. P. D'Urso
A. P. D'Urso 
A.G. Wiley
Alan Wiley
Andy D'Urso
Andy D'Urso 
Andy Hall
B Knight
B. Knight
B. Knight 
Barber, G. P.
Barry Knight
Barry, N. S.
Bennett, S. G.
C Foy
C Kavanagh
C Pawson
C Wilkes
C. J. Foy
C. R. Wilkes
C. R. Wilkes 
Clive Wilkes
D Bond
D Coote
D Elleray
D England
D Gallagh
D Gallaghe
D Gallagher
D Pugh
D'Urso, A. P.
D. J. Gallagher
D. Pugh
D. R. Elleray
D. R. Elleray 
David Ellaray
Dean, M. L
Dermot Gallagher
Dowd, P.
Dunn, S. W.
Durkin, P.
Durkin, P. A.
E Wolstenholme
E. K. Wolstenholme
E. K. Wolstenholme 
Elleray, D. R.
F Taylor
Foy, C. J.
G Barber
G Poll
G Scott
G. P. Barber
G. P. Barber 
G. Poll
G. Poll 
Gallagher, D. J.
Graham Barber
Graham Poll
H Webb
Halsey, M. R.
I Williamson
Ian Harris
J Brooks
J Gillett
J Gillett 
J Moss
J Smith
J Winter
J. T. Winter
J. T. Winter 
J.T. Winter
Jeff Winter
Jones, P.
K Friend
K Stroud
Knight, B.
L Mason
L Probert
L Smith
M